#  Import usefull packages

In [506]:
import spotipy
from IPython.display import IFrame
import pandas as pd
import json
import pprint
import config
from itertools import *
from bs4 import BeautifulSoup as bs
import sys
import random
import requests
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
sp= spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.client_id,client_secret=config.client_secret))

In [507]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from bs4 import BeautifulSoup as bs
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from sklearn.compose import ColumnTransformer

In [508]:
from bs4 import BeautifulSoup as bs
from termcolor import colored
from textblob import TextBlob
import getpass
import difflib
import tkinter as tk
import random

# import the clustered data frame from clustering

In [509]:
db_version_6000= pd.read_csv("db/first_try_6000.csv")

In [510]:
db_version_6000

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,cluster,id
0,0.865,0.665,-6.289,0.0730,0.045300,0.000000,0.2240,0.331,111.113,268827,0,1ofhfV90EnYhEr7Un2fWiv
1,0.801,0.910,-6.566,0.2250,0.002390,0.000891,0.0737,0.805,87.277,278387,0,5g9lS8deSIxItFBmZRC4vN
2,0.644,0.846,-5.443,0.2750,0.124000,0.000000,0.3530,0.503,79.182,301600,2,1qZqFtSINOOmjYH1UE9805
3,0.802,0.787,-6.009,0.2060,0.001270,0.000181,0.0763,0.806,90.189,296907,0,7sKt5Y2zJpYw78iAxndB8y
4,0.872,0.723,-5.968,0.2470,0.029300,0.000000,0.3270,0.843,93.279,307773,0,4VQNCzfZ3MdHEwwErNXpBo
...,...,...,...,...,...,...,...,...,...,...,...,...
6378,0.791,0.591,-9.185,0.0478,0.017200,0.879000,0.1310,0.683,133.974,428606,0,4LWiFuehm2uY3GK8q8sduT
6379,0.655,0.794,-5.872,0.2500,0.082000,0.079100,0.0457,0.667,187.007,184939,0,3cCLvbMCHMgX7DLXPr1ALv
6380,0.757,0.920,-7.697,0.0385,0.000531,0.751000,0.0956,0.955,126.001,327304,0,2MumLWIryjcBG1eRYcRh6I
6381,0.826,0.900,-7.707,0.0697,0.066600,0.275000,0.6790,0.925,120.980,306641,2,5z0J0ztN5QG0UTRv3wSZQK


##  store id and clustr column in other df 

In [511]:
db_version_6000_id=db_version_6000["id"]
db_version_6000_cluster=db_version_6000["cluster"]

## drop id and cluster column 
          we are gonna use db_version_6000

In [512]:
db_version_6000.drop("id",axis=1, inplace=True)
db_version_6000.drop("cluster",axis=1, inplace=True)

## adding them all together again for recommending 

In [513]:
db_version_6000_all= pd.concat([db_version_6000_id,db_version_6000_cluster,db_version_6000],axis=1)

# Load functions

In [514]:
## EMBBEDED SONG PLAYER
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="100",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [515]:
def load(filename = "get_features_df.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!") 

In [516]:
def load(filename = "scaler.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!") 

In [517]:
scaler=load(filename = "Model/scaler.pickle")

In [518]:
# Function to get all tracks from a playlist into a list 
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="DE")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks 
            

In [519]:
def hot_100_song_name():
    song_name_hot_100=[]
    billboard_hot_100=get_playlist_tracks("billboard","6UeSakyzhiEt4NB3UAd6NQ")

    for i in range (0,99):
        a=billboard_hot_100[i]["track"]["name"]
        song_name_hot_100.append(a)
        
    hot_100_song_name=list(map(str.lower,song_name_hot_100))        
           
    return hot_100_song_name

In [520]:
def hot_100():
    song_name_hot_100=[]
    artist_name_hot_100=[]
    id_hot_100=[]
    billboard_hot_100=get_playlist_tracks("billboard","6UeSakyzhiEt4NB3UAd6NQ")

    for i in range (0,99):
        a=billboard_hot_100[i]["track"]["name"]
        song_name_hot_100.append(a)
            
        b=billboard_hot_100[i]["track"]["artists"][0]["name"]
        artist_name_hot_100.append(b)
        c=billboard_hot_100[i]["track"]["id"]
        id_hot_100.append(c)
    
    
    hot_100_song_name=list(map(str.lower,song_name_hot_100))
    hot_100= pd.DataFrame({"title":song_name_hot_100,"artist":artist_name_hot_100,} )  
    
                      
    hot_100["id"]=id_hot_100
    
    return hot_100

In [521]:
hot_100()

,title,artist,id
0,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7
1,Bad Habit,Steve Lacy,5CM4UuQ9Gnd6K2YyKGPMoK
2,Late Night Talking,Harry Styles,1qEmFfgcLObUfQm0j1W2CK
3,Sunroof,Nicky Youre,4h4QlmocP3IuwYEj2j14p8
4,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3
...,...,...,...
94,"Quevedo: Bzrp Music Sessions, Vol. 52",Bizarrap,2tTmW7RDtMQtBk7m2rYeSw
95,Thought You Should Know,Morgan Wallen,6NHpyYvJyQsg2nXXzGYc2R
96,Country On,Luke Bryan,1tRxwf8Q0AcshfHuaD86Yt
97,Static,Steve Lacy,4OmfWzukSVD140NiAIEjem


In [522]:
## KMEANS MODEL from CLUSTERING
def load(filename = "kmeans_6.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!")

In [523]:
kmeans_6=load(filename = "Model/kmeans_6.pickle")

# Define function

1. [x] get the song name from the user as an input
2. [x]  play the input song in music embed player.
3.  [x] get the audio features for that song from spotify API. using sp.audio_features(trackid) . pay attention to keep only the audio features columns.
4.  [x] load the standardscaler using pickle and use it to scale the new song. (scaler.transform)
5. [x] using mode.predict(new scaled audio record for the new song) to predict the cluster (label) for the new song.
6. [x] return random song from the same cluster that the new song belongs to from your data frame and suggest it to the user.
7. [x] play it using embedded music player.  

In [524]:
def song_recommender_():
    welcome = """
************************************************************
*                                                          *
*                  Welcome to the greatest                 *
*                    song recommender!                     *
*                                                          *
************************************************************
"""
    print(welcome)
    
    # INPUT    
    input_song=str.lower((input("Insert a song you like:  ")))
    matches = difflib.get_close_matches(input_song, hot_100_song_name())
    #print(matches)
    #print(len(matches))
    results = sp.search(q=input_song,type="track",market="DE")
    track_id_first_input =results["tracks"]["items"][0]["id"]
    arti=results["tracks"]["items"][0]["artists"][0]["name"]
    songi=results["tracks"]["items"][0]["name"]   
   
    found = hot_100()[hot_100()['id'].str.contains(track_id_first_input)]
    found.count=found["id"].count()
    
   
    
    print("Did you mean this song: ", colored(songi,"blue",attrs=["bold"]), " by ",colored(arti,"green",attrs=["bold"]),"?")
    print()
    print("Press Y for yes or N for no")
    print()

    display(play_song(track_id_first_input))
    accept__= str.lower(input(" "))
    
    if accept__ =="y":
       
        if found.count != 0:
            
            if len(matches)==1:
                results = sp.search(q=matches,type="track",market="DE")
                track_id=results["tracks"]["items"][0]["id"]
                a=results["tracks"]["items"][0]["artists"][0]["name"]
                s=results["tracks"]["items"][0]["name"]      

                display(play_song(track_id))
                print()
                print("Your choosen song is currently in the Bilboard Top 100. \nWe assume that you would also enjoy anoter song from the Billboard Hot 100\n")
                print()

                rec=pd.DataFrame(hot_100().sample().reset_index())
                title=rec["title"].values[0]
                artist= hot_100().loc[hot_100()["title"]==title]["artist"].values[0]
                track_id_rec=rec["id"].values[0]

                print("You should try listening to ",colored(title.capitalize(), 'blue', attrs=['bold'])," from ",colored(artist.capitalize(), 'green', attrs=['bold']),"!")
                print("------------------------------------------------------------------")
                print()
                display(play_song(track_id_rec))


            elif len(matches)>1: 
                print("Do you mean any of the following songs. Enter", colored("Y", "green", attrs=["bold"]), "for yes and ", colored("N","red",attrs=["bold"])," for no")

                for i in range(0, len(matches)):
                    print(colored(matches[i].capitalize(),"blue",attrs=["bold"]))

                input_correct=str.lower(input(""))

                if input_correct == "y":
                    rec=pd.DataFrame(hot_100().sample().reset_index())
                    title=rec["title"].values[0]
                    artist= hot_100().loc[hot_100()["title"]==title]["artist"].values[0]
                    track_id=rec["id"].values[0]

                    print()
                    print("Your choosen song is currently in the Bilboard Top 100. \nWe assume that you would also enjoy another song from Billboard's Hot 100\n")
                    print()
                    print("You should try listening to ", colored(title.capitalize(),"blue",attrs=["bold"])," from ", colored(artist.capitalize(),"green", attrs=["bold"]),"!") 
                    print()
                    print("------------------------------------------------------------------")
                    print()
                    display(play_song(track_id))
        else: 
            #AUDIO FEATURES from User SONG
            user_song_features=sp.audio_features(track_id_first_input)[0]
            df_user_song=pd.DataFrame(user_song_features,index=[0])
            df_user_song=df_user_song[["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms"]]

            # Std Scale the AUDIO FEATURES DF
            scaler.fit(db_version_6000)
            df_user_song_scaled_values = scaler.transform(df_user_song)
            df_user_song_scaled = pd.DataFrame(df_user_song_scaled_values , columns = df_user_song.columns)

            # PREDICT CLUSTER for USER SONG
            kmeans_6.fit(db_version_6000)
            clusters = kmeans_6.predict(df_user_song_scaled)
            cluster_user_song=clusters[0]

            # RANDOM RECOMMENDATION
            sample= db_version_6000_all[db_version_6000_all.cluster==cluster_user_song].sample()
            rec_song_id=sample["id"].values[0]
            #rec_song_name=sample["title"].values[0]
            #rec_song_artist=sample["artist"].values[0]

            # SHOW REC SONG
            #print ("You should try listening to ", colored(rec_song_name,"blue",attrs=["bold"])," from ", colored(rec_song_artist,"green", attrs=["bold"]),"!") 
            print("you should try listening to following song!")
            print()
            print("------------------------------------------------------------------")
            print()
            display(play_song(rec_song_id))
            print()
            print("------------------------------------------------------------------")
   

    else:
        while accept__ =="n":
            for i in range(1, 10):
                track_id=results["tracks"]["items"][i]["id"]
                a=results["tracks"]["items"][i]["artists"][0]["name"]
                s=results["tracks"]["items"][i]["name"]      
                print("Did you mean this song: ", colored(s,"blue",attrs=["bold"]), " by ",colored(a,"green",attrs=["bold"]),"?")
                print()
                print("Press Y for yes or N for no")
                print()

                display(play_song(track_id))
                accept__= str.lower(input(" "))
                if accept__ == "y":
                    break

        #AUDIO FEATURES from User SONG
        user_song_features=sp.audio_features(track_id)[0]
        df_user_song=pd.DataFrame(user_song_features,index=[0])
        df_user_song=df_user_song[["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms"]]

        # Std Scale the AUDIO FEATURES DF
        scaler.fit(db_version_6000)
        df_user_song_scaled_values = scaler.transform(df_user_song)
        df_user_song_scaled = pd.DataFrame(df_user_song_scaled_values , columns = df_user_song.columns)

        # PREDICT CLUSTER for USER SONG
        kmeans_6.fit(db_version_6000)
        clusters = kmeans_6.predict(df_user_song_scaled)
        cluster_user_song=clusters[0]

        # RANDOM RECOMMENDATION
        sample= db_version_6000_all[db_version_6000_all.cluster==cluster_user_song].sample()
        rec_song_id=sample["id"].values[0]
        #rec_song_name=sample["title"].values[0]
        #rec_song_artist=sample["artist"].values[0]

        # SHOW REC SONG
        #print ("You should try listening to ", colored(rec_song_name,"blue",attrs=["bold"])," from ", colored(rec_song_artist,"green", attrs=["bold"]),"!") 
        print("you should try listening to following song!")
        print()
        print("------------------------------------------------------------------")
        print()
        display(play_song(rec_song_id))
        print()
        print("------------------------------------------------------------------")

    return 

# Save function

In [525]:
with open("Model/song_recommender_.pickle", "wb") as f:
    pickle.dump(song_recommender_,f)

# RECOMMEND 


In [526]:
song_recommender_()


************************************************************
*                                                          *
*                  Welcome to the greatest                 *
*                    song recommender!                     *
*                                                          *
************************************************************

Insert a song you like:  cuff it
Did you mean this song:  CUFF IT  by  Beyoncé ?

Press Y for yes or N for no



 y
you should try listening to following song!

------------------------------------------------------------------




------------------------------------------------------------------
